In [1]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By
import random
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException

# Khởi tạo trình duyệt
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.foody.vn/ho-chi-minh/nha-hang-san-fu-lou/binh-luan")
sleep(random.randint(5, 10))

# Lấy số lượng bình luận và số lần click "Xem thêm bình luận"
try:
    review_count = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[2]/section/div/div/div/div/div[1]/div/div/div[1]/div/div[1]/div/ul/li[1]/a/span")
    review_count = int(review_count.text.replace(',', ''))
    click_count = review_count // 10
except NoSuchElementException:
    print("Không thể lấy số lượng bình luận")
    driver.quit()
    exit()

# Click "Xem thêm bình luận"
for i in range(click_count):
    try:
        showcomment_link = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[2]/section/div/div/div/div/div[1]/div/div/div[1]/div/div[2]/a")
        showcomment_link.click()
        sleep(random.randint(5, 10))
    except NoSuchElementException:
        print(f"No Such Element Exception tại lần click thứ {i}")
        break

# Lấy tên và link tài khoản bình luận
elems = driver.find_elements(By.CSS_SELECTOR, ".ru-row [href]")
account_names = [elem.text for elem in elems]
account_links = [elem.get_attribute('href') for elem in elems]

df_accounts = pd.DataFrame(list(zip(account_names, account_links)), columns=['Tên tài khoản', 'Link tài khoản'])

# Hàm lấy số hoạt động từ tài khoản
def get_activities(df, index):
    activities_data = {
        'Tên tài khoản': [],
        'Các địa điểm đã đến': [],
        'Ngày Giờ review': [],
        'Công cụ review': [],
        'Chấm điểm': [],
        'Nội dung bình luận': [],
        'Số lượt xem nội dung bình luận': []
    }
    try:
        activity = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div/div/div[1]/div/div/ul/li/span").text
        activity_count = int(activity)
        
        # Click nút "Xem thêm" nếu có
        if activity_count > 10:
            for _ in range((activity_count - 10) // 10 + 1):
                try:
                    view_more_activities = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div/div/div[2]/div/a")
                    view_more_activities.click()
                    sleep(random.randint(1, 3))
                except ElementNotInteractableException:
                    break
        
        # Lấy thông tin các hoạt động
        for k in range(activity_count):
            try:
                restaurant_name = driver.find_element(By.XPATH, f"/html/body/div[2]/div[6]/div/div[2]/div/div/div[2]/ul/li[{k + 1}]/div[1]/div[2]/div[2]/a").text
                review_datetime = driver.find_element(By.XPATH, f"/html/body/div[2]/div[6]/div/div[2]/div/div/div[2]/ul/li[{k + 1}]/div[1]/div[2]/div[3]/span[1]").text
                review_tool = driver.find_element(By.XPATH, f"/html/body/div[2]/div[6]/div/div[2]/div/div/div[2]/ul/li[{k + 1}]/div[1]/div[2]/div[3]/a").text
                rating_score = driver.find_element(By.XPATH, f"/html/body/div[2]/div[6]/div/div[2]/div/div/div[2]/ul/li[{k + 1}]/div[1]/div[3]/span").text
                comment_text = driver.find_element(By.XPATH, f"/html/body/div[2]/div[6]/div/div[2]/div/div/div[2]/ul/li[{k + 1}]/div[2]/div/span").text
                view_count = driver.find_element(By.XPATH, f"/html/body/div[2]/div[6]/div/div[2]/div/div/div[2]/ul/li[{k + 1}]/div[1]/div[2]/div[3]/span[2]").text
                
                # Lưu thông tin vào danh sách
                activities_data['Tên tài khoản'].append(df['Tên tài khoản'][index])
                activities_data['Các địa điểm đã đến'].append(restaurant_name)
                activities_data['Ngày Giờ review'].append(review_datetime)
                activities_data['Công cụ review'].append(review_tool)
                activities_data['Chấm điểm'].append(rating_score)
                activities_data['Nội dung bình luận'].append(comment_text)
                activities_data['Số lượt xem nội dung bình luận'].append(view_count)
                
            except NoSuchElementException:
                break
    except NoSuchElementException:
        df.loc[index, 'activity_count'] = "Không có hoạt động"
    
    return pd.DataFrame(activities_data)

# Tạo dataframe lưu các hoạt động của tài khoản
all_activities = pd.DataFrame(columns=['Tên tài khoản', 'Các địa điểm đã đến', 'Ngày Giờ review', 'Công cụ review', 'Chấm điểm', 'Nội dung bình luận', 'Số lượt xem nội dung bình luận'])

# Duyệt qua các tài khoản và thu thập thông tin hoạt động
for i, account_link in enumerate(account_links):
    driver.get(account_link)
    sleep(random.randint(5, 10))
    activities_df = get_activities(df_accounts, i)
    all_activities = pd.concat([all_activities, activities_df], ignore_index=True)

# Lưu kết quả vào file CSV
all_activities.to_csv("binhluan1.csv", index=False, encoding="utf-8-sig")

# Đóng trình duyệt
driver.quit()


AttributeError: 'str' object has no attribute 'capabilities'